In [1]:
%%html
<style>
li,p {
    font-size: 18px;
}
</style>    

In [ ]:
import numpy as np
np.random.seed(42)

import holoviews as hv
hv.notebook_extension('matplotlib')

%opts Points [color_index=2] (cmap="bwr" edgecolors='k' s=50 alpha=1.0)
%opts Scatter3D [color_index=3 fig_size=250] (cmap='bwr' edgecolor='k' s=50 alpha=1.0)
%opts Image (cmap="gray_r") {+axiswise}
%opts RGB [bgcolor="black" show_grid=False]

import holoviews.plotting.mpl
holoviews.plotting.mpl.MPLPlot.fig_alpha = 0
holoviews.plotting.mpl.ElementPlot.bgcolor = 'white'

from holoviews.operation.datashader import datashade
import colorcet as cc


In [ ]:
try:
    from skimage.exposure import equalize_hist
    eq_hist = lambda d,m: equalize_hist(1000*d,nbins=100000,mask=m)
except ImportError:
    eq_hist = lambda d,m: d
    print("scikit-image not installed; skipping histogram equalization")

def heatmap(coords,bins=10,offset=0.0,transform=lambda d,m:d, label=None):
    """
    Given a set of coordinates, bins them into a 2d histogram grid
    of the specified size, and optionally transforms the counts
    and/or compresses them into a visible range starting at a 
    specified offset between 0 and 1.0.
    """
    hist,xs,ys  = np.histogram2d(coords[0], coords[1], bins=bins)
    counts      = hist[:,::-1].T
    transformed = transform(counts,counts!=0)
    span        = transformed.max()-transformed.min()
    compressed  = np.where(counts!=0,offset+(1.0-offset)*transformed/span,0)
    args        = dict(label=label) if label else {}
    return hv.Image(compressed,bounds=(xs[-1],ys[-1],xs[1],ys[1]),**args)

def gaussians(specs=[(1.5,0,1.0),(-1.5,0,1.0)],num=100):
    """
    A concatenated list of points taken from 2D Gaussian distributions.
    Each distribution is specified as a tuple (x,y,s), where x,y is the mean
    and s is the standard deviation.  Defaults to two horizontally
    offset unit-mean Gaussians.
    """
    np.random.seed(1)
    dists = [(np.random.normal(x,s,num), np.random.normal(y,s,num)) for x,y,s in specs]
    return np.hstack([d[0] for d in dists]), np.hstack([d[1] for d in dists])

## What is Datashader?
- Makes pictures of large datasets, fast!
- Preserves distribution and outliers in the visualization
- Highly performant - can plot a billion points on a laptop
- Not exactly a plotting library but plays well with HoloViews and Bokeh (part of the PyViz ecosystem)
- http://datashader.org

## When would I want to use it?
- When you have a LOT of data to plot, like tens of thousands or more points
    - Can handle a billion points on a laptop with 16GB of RAM, more from a cluster
- When you might be tempted to use a sample to plot with another library

# Synthetic Example of Plotting Pitfalls

- A bunch of points from 5 different gaussian distributions 
- 4 clusters of different sizes at the corners of a square, and one big cluster that overlaps all of them
- This example comes from the 'Plotting Pitfalls' section of the Datashader User Guide http://datashader.org/user_guide/1_Plotting_Pitfalls.html

In [ ]:
%%opts Layout [sublabel_format="" tight=True] Points {-axiswise}
hv.notebook_extension('matplotlib')

dist = gaussians(specs=[(2,2,0.02), (2,-2,0.1), (-2,-2,0.5), (-2,2,1.0), (0,0,3)],num=10000)

(hv.Points(dist,label="1. Overplotting") + 
 hv.Points(dist,label="2. Oversaturation")(style=dict(s=0.1,alpha=0.5)) + 
 hv.Points((dist[0][::200],dist[1][::200]),label="3. Undersampling")(style=dict(s=2,alpha=0.5))).cols(3)

In [ ]:
%%opts Layout [sublabel_format="" tight=True] Points {-axiswise}
hv.notebook_extension('matplotlib')
(hv.Points(dist,label="4. Undersaturation")(style=dict(s=0.01,alpha=0.05)) + 
heatmap(dist,200,offset=0.2,label="5. Underutilized dynamic range") +
heatmap(dist,200,transform=eq_hist,label="6. Nonuniform colormapping")(style=dict(cmap="hot"))).cols(3)

## Datashader just works to plot the best image
- Given the available pixels on the screen!

In [ ]:
%output size=200
datashade.cmap=cc.rainbow[50:]
datashade(hv.Points(dist))

In [ ]:
dist = gaussians(specs=[(2,2,0.02), (2,-2,0.1), (-2,-2,0.5), (-2,2,1.0), (0,0,3)],num=5000000)

datashade(hv.Points(dist))

# Under the hood
- Extensible processing Pipeline
- Render data into a screen sized aggregate array
- It sends an image back to the browser
- Data doesn't have to all be in memory or on the same machine
- Uses Numba and Dask for performance

![Datashader Pipeline](http://datashader.org/assets/images/pipeline2.png)

# Real Data
This is a dataset consisting of over 10 million taxi trips in NYC

In [ ]:
data_dir = "../datashader-examples/data/"

In [ ]:
import geoviews as gv
import pandas as pd

hv.extension('bokeh', width=95)

plot_width= 700
plot_height = int(plot_width//1.2)

%opts RGB     [width=plot_width, height=plot_height, xaxis=None yaxis=None show_grid=False] 
%opts Shape (fill_alpha=0 line_width=1.5) [apply_ranges=False tools=['tap']] 
%opts Points [apply_ranges=False] WMTS (alpha=0.5)

datashade.cmap=cc.fire[50:]

In [ ]:
df = pd.read_csv(data_dir + 'nyc_taxi.csv', usecols=\
                                ['pickup_x', 'pickup_y', 'dropoff_x','dropoff_y', 'passenger_count','tpep_pickup_datetime'])

taxi_points = hv.Points(df, kdims=['pickup_x', 'pickup_y'])
len(df)

In [ ]:
shaded1 = datashade(taxi_points)

In [ ]:
tiles = gv.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg')
tiles * shaded1

# Zooming

When connected to a server, Datashader will re-render points when zooming.

![Zoom Example](zoom.png)

# Some other things Datashader can do
- Timeseries plots
- More advanced geospatial plots
- Trajectories, e.g. from GPS
- Triangle meshes to render polygons